<a href="https://colab.research.google.com/github/ekaterinaKrupkina/decomposer_notebooks/blob/main/separate_steps/experiments__evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
import subprocess
import json
from google.colab import userdata
import shlex
import pandas as pd
import time
import sys
from datetime import datetime


## Set up environment

## !!! After installation you'll be asked to reload the runtime. Please, REFUSE

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
REQUIREMENTS_PATH = "/content/drive/MyDrive/HarmBench/decomposer_notebooks/requirements_full.txt"

In [ ]:
!pip install -r {REQUIREMENTS_PATH}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.7/328.7 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.5/865.5 kB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.8/601.8 kB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.9/29.9 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
%cd /content/drive/MyDrive/HarmBench

/content/drive/MyDrive/HarmBench


### Set variables

* WANDB_API_KEY
* GITHUB_TOKEN
* OPENAI_API_KEY
* MISTRAL_API_KEY
* CLAUDE_API_KEY
* HF_TOKEN

In [ ]:
# wandb & badlamma setting
os.environ["WANDB__EXECUTABLE"] = "/usr/local/bin/python"

In [ ]:
def set_env_variable(variable_name):
    """
    Sets an environment variable with the specified name. If the variable is already present
    in the environment, it uses the existing value from `userdata`. If not, it prompts the user
    to input the value and sets the environment variable with that input.

    Args:
        variable_name (str): The name of the environment variable to be set.

    Raises:
        KeyError: If the variable_name is not present in `userdata` when it is expected.
    """
    if userdata.get(variable_name):
       os.environ[variable_name] = userdata.get(variable_name)
    else:
       os.environ[variable_name] = input(f"Enter the value for {variable_name}: ")

In [ ]:
set_env_variable("OPENAI_API_KEY")
set_env_variable("WANDB_API_KEY")
set_env_variable("GITHUB_TOKEN")

## Functions

In [ ]:
def run_evaluate_completions(experiment_name, behaviors_path):
    """
    Run the evaluate_completions.py script as a subprocess.

    This function executes the evaluate_completions.py script with the specified parameters,
    captures its output in real-time, and prints it to the console.

    Args:
        behaviors_path (str): Path to the behaviors dataset file.
        completions_path (str): Path to the completions file to be evaluated.
        save_path (str): Path where the evaluation results will be saved.

    Returns:
        None

    Raises:
        subprocess.CalledProcessError: If the subprocess returns a non-zero exit status.
    """

    # Set up variables

    completions_path = f'/content/drive/MyDrive/HarmBench/results/completions/completions_{experiment_name}.json'
    save_path = f'/content/drive/MyDrive/HarmBench/results/evaled_{experiment_name}.json'

    # Execute the script as a subprocess
    command = [
        sys.executable,
        "-u",
        "evaluate_completions.py",
        f"--behaviors_path={behaviors_path}",
        f"--completions_path={completions_path}",
        f"--save_path={save_path}"
    ]

    try:
        process = subprocess.Popen(
            command,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )

        for line in process.stdout:
            print(line, end='')

    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")

In [ ]:
import wandb

def process_results(experiment_name, project="test_db", entity="palisaderesearch"):
    """
    Process the results of the evaluation and log them to Weights & Biases.

    This function reads the evaluation results from a JSON file, creates a DataFrame,
    and logs the results to Weights & Biases for tracking and visualization.

    Args:
        file (str): Name of the JSON file containing the evaluation results (without .json extension).
        decomposer (bool): Flag indicating whether a decomposer was used in the experiment.
        config_file (str): Path to the JSON config file to append to the run's config.

    Returns:
        None
    """
    # Set up variables
    MODEL_TARGET = os.getenv("MODEL_TARGET")
    MODEL_DECOMPOSER = os.getenv("MODEL_DECOMPOSER")
    config_file=f'/content/drive/MyDrive/HarmBench/config_{experiment_name}.json'
    evaled_file = f'/content/drive/MyDrive/HarmBench/results/evaled_{experiment_name}.json'

    # Load the JSON data
    data_eval = pd.read_json(evaled_file)
    data_summ = pd.read_json(f"/content/drive/MyDrive/HarmBench/summary_{MODEL_TARGET}_{MODEL_DECOMPOSER}.json")

    # Convert JSON data to DataFrame
    df_evaled = pd.DataFrame([{"category": key, **value[0]} for key, value in data_eval.items()])
    df = df_evaled.merge(data_summ, on='test_case', how='inner')

    # Add additional columns
    df["target_model"] = os.getenv("MODEL_TARGET")
    df["decomposer_model"] = os.getenv("MODEL_DECOMPOSER")
    df['composer_model'] = os.getenv("MODEL_COMPOSER")
    df["surrogate_model"] = os.getenv("MODEL_SURROGATE")
    df["ASR"] = df["label"].mean()

    # Remove duplicates
    df = df.drop_duplicates(subset=['test_case'])

    # Remove the 'generation_y' column
    df.drop('generation_y', axis=1, inplace=True)

    # Rename 'generation_x' to 'generation'
    df.rename(columns={'generation_x': 'generation'}, inplace=True)

    # Rename columns to start with 'tokens_' or 'price_'
    rename_dict = {
        'surrogate_tokens': 'tokens_surrogate',
        'target_tokens': 'tokens_target',
        'decomposer_tokens': 'tokens_decomposer',
        'composer_tokens': 'tokens_composer',
        'surrogate_price': 'price_surrogate',
        'target_price': 'price_target',
        'decomposer_price': 'price_decomposer',
        'composer_price': 'price_composer'
    }
    df.rename(columns=rename_dict, inplace=True)

    # Print the DataFrame
    print(df.head())

    # Initialize W&B run
    run = wandb.init(
        entity=entity,
        project=project,
        save_code=False,
        name=f"decomp_{MODEL_TARGET}_{MODEL_DECOMPOSER}_full"
    )

    # Load the config file and update run config
    with open(config_file, 'r') as f:
        config_data = json.load(f)
        run.config.update(config_data)

    # Log the DataFrame as a W&B Table
    run.log({"harmbench": wandb.Table(dataframe=df)})
    run.summary["harmbench_mean"] = df["label"].mean()
    run.summary["target_model"] = os.getenv("MODEL_TARGET")

    # Finish the W&B run
    run.finish()


In [ ]:
def experiment(experiment_name, experiment_params, project='test_db'):
    """
    Run a complete experiment pipeline including completion generation, evaluation, and result processing.

    This function sets up the environment, runs the necessary scripts for generating completions
    and evaluating them, and then processes and logs the results.

    Args:
        experiment_params (dict): A dictionary containing experiment parameters including model names.

    Returns:
        None
    """

    # Set up environment variables
    for k, v in experiment_params.items():
        os.environ[k] = str(v)

    try:
      run_evaluate_completions(
          experiment_name,
          behaviors_path="./data/behavior_datasets/harmbench_behaviors_text_all.csv"
      )

      process_results(experiment_name, project)

    except Exception as e:
      print(e)
      pass

    for k, v in experiment_params.items():
      del os.environ[k]



In [ ]:
def get_summary_table(number_of_runs, project_name, entity_name='palisaderesearch'):
    api = wandb.Api()

    # Get all runs for the specified project
    runs = api.runs(f"{entity_name}/{project_name}")

    # Sort runs by creation time (newest first)
    sorted_runs = sorted(runs, key=lambda run: datetime.strptime(run.created_at, "%Y-%m-%dT%H:%M:%S"), reverse=True)

    # Get the n newest run IDs
    newest_run_ids = [run.id for run in sorted_runs[:number_of_runs]]

    full_table = pd.DataFrame()

    # Iterate over the newest run IDs
    for run_id in newest_run_ids:
        artifact_name = f"run-{run_id}-harmbench:v0"
        artifact_full_name = f"{entity_name}/{project_name}/{artifact_name}"

        try:
            # Attempt to retrieve the artifact
            artifact = api.artifact(artifact_full_name)
            table = artifact.get("harmbench")
            df = pd.DataFrame(data=table.data, columns=table.columns)

            # Aggregate the data
            agg_cols = ['price_total', 'tokens_total', 'ASR']
            group_by_cols = ['target_model', 'decomposer_model']
            aggregated_df = df.groupby(group_by_cols)[agg_cols].mean().reset_index()

            # Rename the columns
            aggregated_df = aggregated_df.rename(columns={
                'price_total': 'AVG_price',
                'tokens_total': 'AVG_tokens'
            })

            # Add run_id column
            aggregated_df['run_id'] = run_id

            full_table = pd.concat([full_table, aggregated_df], ignore_index=True)

        except wandb.errors.CommError:
            print(f"Artifact not found: {artifact_full_name}")

    # Initialize a new wandb run
    with wandb.init(project=project_name, entity=entity_name, job_type="summary_table") as run:
        # Log the summary table
        run.log({"summary_table": wandb.Table(dataframe=full_table)})

    # Print the resulting table
    print("Summary Table:")
    print(full_table)

    return full_table

## Run experiments

In [ ]:
# path to experiments
file_path = '/content/drive/MyDrive/HarmBench/experiment_params/experiments.json'

In [ ]:
with open(file_path, 'r') as json_file:
    experiments_dict = json.load(json_file)

In [ ]:
for exp, params in experiments_dict.items():
    print(f"{exp}: {params}")


In [ ]:
for experiment_name, experiment_params in experiments_dict.items():
  print(experiment_name)
  experiment(experiment_params)
  print('======================================')
print("===================END=======================")

In [ ]:
# clean cash if needed

import gc
import torch

gc.collect()

torch.cuda.empty_cache()

## Push summary table to Wandb



In [ ]:
get_summary_table(5, 'test_db')



---

end of file